<a href="https://colab.research.google.com/github/satria080503/agriculture-with-AI/blob/main/Crop_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df = pd.read_csv('Crop_recommendation.csv')
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [6]:
df['label'] = pd.Categorical(df['label']).codes
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,20
1,85,58,41,21.770462,80.319644,7.038096,226.655537,20
2,60,55,44,23.004459,82.320763,7.840207,263.964248,20
3,74,35,40,26.491096,80.158363,6.980401,242.864034,20
4,78,42,42,20.130175,81.604873,7.628473,262.717340,20


In [7]:
X = df.drop(['label'],axis=1).values
y = df['label'].values

print(X.shape, y.shape)

(2200, 7) (2200,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_raw, y_train, y_raw = train_test_split(X, y, test_size=0.4)
X_test, X_val, y_test, y_val = train_test_split(X_raw, y_raw, test_size=0.3)

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)
print('Val', X_val.shape, y_val.shape)

Train (1320, 7) (1320,)
Test (616, 7) (616,)
Val (264, 7) (264,)


In [9]:
import keras

inputs = keras.Input((7,))

x = keras.layers.Reshape((7, 1))(inputs)
x = keras.layers.LSTM(48)(x)
x = keras.layers.Dense(24, activation="gelu")(x)
outputs = keras.layers.Dense(22, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7)]               0         
                                                                 
 reshape (Reshape)           (None, 7, 1)              0         
                                                                 
 lstm (LSTM)                 (None, 48)                9600      
                                                                 
 dense (Dense)               (None, 24)                1176      
                                                                 
 dense_1 (Dense)             (None, 22)                550       
                                                                 
Total params: 11326 (44.24 KB)
Trainable params: 11326 (44.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test));

Epoch 1/50
42/42 [==============================] - 3s 22ms/step - loss: 3.0797 - accuracy: 0.0455 - val_loss: 3.0407 - val_accuracy: 0.1169
Epoch 2/50
42/42 [==============================] - 0s 7ms/step - loss: 2.9707 - accuracy: 0.1364 - val_loss: 2.8228 - val_accuracy: 0.1640
Epoch 3/50
42/42 [==============================] - 0s 8ms/step - loss: 2.6283 - accuracy: 0.1932 - val_loss: 2.3430 - val_accuracy: 0.2321
Epoch 4/50
42/42 [==============================] - 0s 8ms/step - loss: 2.1024 - accuracy: 0.3644 - val_loss: 1.7924 - val_accuracy: 0.5276
Epoch 5/50
42/42 [==============================] - 0s 8ms/step - loss: 1.5952 - accuracy: 0.5758 - val_loss: 1.3803 - val_accuracy: 0.6494
Epoch 6/50
42/42 [==============================] - 0s 7ms/step - loss: 1.2520 - accuracy: 0.6902 - val_loss: 1.1023 - val_accuracy: 0.7192
Epoch 7/50
42/42 [==============================] - 0s 8ms/step - loss: 1.0427 - accuracy: 0.7402 - val_loss: 0.9740 - val_accuracy: 0.7451
Epoch 8/50
42/42 [=

In [11]:
import tensorflow.keras.backend as K
y_pred = K.argmax(model.predict(X_val), axis=1)

9/9 [==============================] - 0s 3ms/step


In [12]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 97.73%


In [13]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.93      1.00      0.97        14
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        22
           4       1.00      1.00      1.00        16
           5       1.00      1.00      1.00        11
           6       1.00      1.00      1.00        16
           7       1.00      1.00      1.00        11
           8       0.70      0.88      0.78         8
           9       1.00      1.00      1.00         8
          10       0.89      1.00      0.94         8
          11       1.00      1.00      1.00        13
          12       1.00      1.00      1.00        10
          13       1.00      0.93      0.96        14
          14       1.00      1.00      1.00        20
          15       1.00      1.00      1.00         7
          16       1.00      1.00      1.00         7
          17       1.00    

In [24]:
import numpy as np
# Test the model using data
data = [[90,42,43,20,81,7,210]]  # Example data point

# Make predictions
predictions = model.predict(data)


predicted_class = np.argmax(predictions)

crop_names = ['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee',
              'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize',
              'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya',
              'pigeonpeas', 'pomegranate', 'rice', 'watermelon']

# Create a new dictionary with numeric keys based on the order
crop_dict = {i: crop_names[i] for i in range(len(crop_names))}

if predicted_class in crop_dict:
    crop = crop_dict[predicted_class]
    print(f"Predicted class: {predicted_class} ({crop})")
    print(f"{crop} is a possible crop to be cultivated.")
else:
    print(f"Predicted class: {predicted_class}")
    print("Sorry, we are not able to recommend a proper crop for this environment based on the model's prediction.")


1/1 [==============================] - 0s 474ms/step
Predicted class: 20 (rice)
rice is a possible crop to be cultivated.


In [25]:
import pickle
pickle.dump(model,open('model.pkl','wb'))